# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$


### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo 

In [14]:
import numpy as np
from numpy.linalg import det, inv

In [15]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    
    # Q1: why is there no need for class_to_name?
    # Porque con el diccionario creado se puede acceder por la clave o el valor
    #  Ej. 
    #     print(list(self.name_to_class.values())[0])
    #     print(list(self.name_to_class.keys())[0])

  def _map_reshape(self, f, arr):

    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)

    # Q2: why is reshaping necessary?
    # Para poder recorrer todos los elementos de arr se convierte a la matriz en un vector fila mediante arr.flatten. 
    # Para poder reestablecer el formato se usa reshape  

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [16]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    
    # Q3: what does bincount do?
    # Cuenta la aparición de cada elemento de cada clase.
    # En este caso cuenta cuantas veces aparece cada elemento 'y' del dataset.
    # Por ej. cuantas 'setosa','virginica', etc. hay.

    return np.log(a_priori)
    
  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes

    y = self.encoder.fit_transform(y)
    
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    
    # Q4: why do we need to do this last, can't we do it first?
    # No se puede hacer inmediatamente despues de "def fit" porque _fit_params necesita tener resuelto self.log_a_priori 

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]

    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
    
    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]
           
    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i 
                  in enumerate(self.log_a_priori) ]
    
    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [17]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    
    self.inv_covs = [inv(np.cov(X[:, y.flatten()==idx], bias=True)) 
                      for idx in range(len(self.log_a_priori))]
    
    # Q5: why not just X[:,y==idx]?
    # 'y' es una matriz pero se requiere un vector para usar como indice de columnas de X
    
    # Q6: what does bias=True mean? why not use bias=False?
    # np.cov estima la matriz de covarianza. 
    # Con bias=True la normalización se hace con todos los valores de N. Si bias=False la normalización es por (N - 1) , 
    # donde N es el número de observaciones dadas (estimación insesgada). 

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    
    # Q7: what does axis=1 mean? why not axis=0 instead?
    # axis=1 significa que el arreglo se recorre por columnas. axis=0 es por filas. Cada columna de X tienen la 
    # descripcion de los atributos de una flor por eso hay que recorrer columna por columna.
    
  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x
    

### NUEVO CÓDIGO ###

In [18]:
class QDA1(QDA):

  def _estimate_a_priori(self, y):
    a_priori = np.ones([len(self.encoder.names)], dtype = int) * 1/len(self.encoder.names)

    return np.log(a_priori)

In [19]:
class LDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
                      
    #self.inv_covs = [inv(np.cov(X[:, y.flatten()==0], bias=True)) 
    #                  for idx in range(len(self.log_a_priori))]
    
    #inv_cov_mean = np.mean([np.cov(X[:, y.flatten()==idx], bias=True)
    #                  for idx in range(len(self.log_a_priori))])
    
    #self.inv_cov = inv(inv_cov_mean)
      
    #a = np.array([[1, 2], [2, 4]])
    # np.mean([a for i in range(3)], axis=0)
    
    self.inv_cov = inv(np.cov(X[:, y.flatten()==0], bias=True)) 
    
    # print('self.inv_covs : \n',self.inv_cov)

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    
    # print('self.means : \n', self.means)

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_cov
    unbiased_x =  x - 0.5 * self.means[class_idx]
    
    # retorna el log de la prediccion de acuerdo a la formula para LDA
    return self.means[class_idx].T @ inv_cov @ unbiased_x


## Código para pruebas

In [20]:
# hiperparámetros
#rng_seed =  6543
#rng_seed =  1234
#rng_seed =  2
rng_seed =  9347

In [21]:
from sklearn.datasets import load_iris

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [22]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [23]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

In [24]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.4, random_state=rng_seed)

# traspose everything because this model format assumes column vectors
train_x = X_train.T
train_y = y_train.T
test_x = X_test.T
test_y = y_test.T

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


In [25]:
qda = QDA()
#qda = QDA1()
#qda = LDA()

qda.fit(train_x, train_y)

In [26]:
def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


# Consigna

## Implementación
1. Entrenar un modelo QDA utilizando ahora una *a priori* uniforme ¿Se observan diferencias?¿Por qué?

Respuesta:

El resultado es : Train (apparent) error is 0.0222 while test error is 0.0167

Se ve que asumiendo una distribución uniforme el error en test es mayor que el mismo error tomando la frecuencia relativa. Porque el método asume una distribución uniforme. 

En test los resultados son iguales para ambas a_priori. 

2. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?

LDA da : Train (apparent) error is 0.0333 while test error is 0.0167

En train el error se triplicó pero en test no se aprecian diferencias.

3. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?

Los valores utilizados son : \
Con LDA : \
rng_seed =  6543 : Train (apparent) error is 0.0333 while test error is 0.0167 \
rng_seed =  1234 : Train (apparent) error is 0.0333 while test error is 0.0167 \
rng_seed =  2    : Train (apparent) error is 0.0222 while test error is 0.0667 \
rng_seed =  9347 : Train (apparent) error is 0.0444 while test error is 0.0333 

Con QDA : \
rng_seed =  6543 : Train (apparent) error is 0.0111 while test error is 0.0167 \
rng_seed =  1234 : Train (apparent) error is 0.0222 while test error is 0.0000 \
rng_seed =  2    : Train (apparent) error is 0.0333 while test error is 0.0167 \
rng_seed =  9347 : Train (apparent) error is 0.0111 while test error is 0.0167 


1. *(Opcional)* En `BaseBayesianClassifier._predict_one` se estima la posteriori de cada clase por separado, a pesar de que la cuenta es siempre la misma (cambiando valores de parámetros, pero no dimensiones). Aprovechando el *broadcasting* de NumPy, se puede reemplazar ese list comprehension por un cálculo *tensorizado* donde tanto medias como varianzas (o inversas) estén "stackeadas" sobre un tercer eje, permitiendo el cálculo en paralelo de todas las clases con un:
`log_posteriori = self.tensor_log_a_priori + self._predict_log_conditionals(x)`. Implementar dicha modificación y mostrar su uso. *Ayuda: los métodos `np.stack` y la documentación del operador [`@`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) son de gran utilidad.*

## Preguntas técnicas

Responder las 7 preguntas que se encuentran distribuidas a lo largo del código.

Respuesta : Ver código

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.
--------------------------------------------------------------------------------------
Respuesta :


$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva. 

Partimos de la ecuación $\log{f_j(x)} =  - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C'$

$$
- \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) = -\frac{1}{2} (x^T \Sigma^{-1} x - x^T \Sigma^{-1} \mu_j - \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j) 
$$

$(x^T \Sigma^{-1} \mu_j)$ es un número real -----> $x^T \Sigma^{-1} \mu_j = (x^T \Sigma^{-1} \mu_j)^T$, ademas $\Sigma$ es simetrica, entonces $\Sigma^{-1} = (\Sigma^{-1})^T$

Por lo anterior :  $x^T \Sigma^{-1} \mu_j = \mu_j^T  \Sigma^{-1}  x $

$$
- \frac{1}{2} (x^T \Sigma^{-1} x - x^T \Sigma^{-1} \mu_j - \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j) = - \frac{1}{2} (x^T \Sigma^{-1} x - \mu_j^T \Sigma^{-1} x - \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j) = -\frac{1}{2} (x^T \Sigma^{-1} x - 2 \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j) 
$$

$ (x^T \Sigma^{-1} x)$ no depende de la población, por lo cual podemos quitarla de la comparación

$$
-\frac{1}{2} (x^T \Sigma^{-1} x - 2 \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j) = \mu_j^T \Sigma^{-1} x -\frac{1}{2} (\mu_j^T \Sigma^{-1} \mu_j) 
$$
$$
= \mu_j^T \Sigma^{-1} (x-\frac{1}{2} \mu_j)
$$

------------------------------------------------------------------

2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".


Respuesta :
Cuando la covarianza se asume igual en todas las clases se simplifica el termino cuadratico y queda una ecuación lineal:

$$
\log{f_1(x)} - \log{f_2(x)}=  -\frac{1}{2} (x^T \Sigma_1^{-1} x - 2 \mu_1^T \Sigma_1^{-1} x + \mu_1^T \Sigma_1^{-1} \mu_1) - ( -\frac{1}{2} (x^T \Sigma_2^{-1} x - 2 \mu_2^T \Sigma_2^{-1} x + \mu_2^T \Sigma_2^{-1} \mu_2))
$$

$$
= -\frac{1}{2} [(x^T \Sigma_1^{-1} x - 2 \mu_1^T \Sigma_1^{-1} x + \mu_1^T \Sigma_1^{-1} \mu_1) - (x^T \Sigma_2^{-1} x - 2 \mu_2^T \Sigma_2^{-1} x + \mu_2^T \Sigma_2^{-1} \mu_2)]
$$


Podemos separar la acuación en terminos cuadraticos ($x^{2}$), lineales (x) e independiente (no contiene x): 
$$
= -\frac{1}{2} x^T (\Sigma_1^{-1} - \Sigma_2^{-1}) x + (\mu_1^T \Sigma_1^{-1} - \mu_2^T \Sigma_2^{-1})x-\frac{1}{2}(\mu_1^T \Sigma_1^{-1} \mu_1 - \mu_2^T \Sigma_2^{-1} \mu_2)
$$

Se puede apreciar que si las covarianzas son iguales en todos las clases el termino $x^T (\Sigma_1^{-1} - \Sigma_2^{-1}) x $ queda como $x^T (\Sigma^{-1} - \Sigma^{-1}) x $ y desparece de la ecuación, pasando de una ecuación cuadratica (QDA) a una ecuación lineal (LDA)

---------------------------------------------------------------------

3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

Respuesta :

La ecuación del apartado teorico es : $\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C$

Analizando cada uno de los terminos se ve que la diferencia está en el termino '0.5*np.log(det(inv_cov))'

0.5 * np.log(det(inv_cov)) = $\frac{1}{2} \log |\Sigma^{-1}|$ =? $-\frac{1}{2}\log |\Sigma|$

Considerando que el determinante de la inversa de una matriz es igual a la inversa del determinate de la matriz se puede hacer : 

$\frac{1}{2}\log |\Sigma^{-1}|= \frac{1}{2}\log (\frac{1}{|\Sigma|})=\frac{1}{2}(\log(1) - log |\Sigma|) = \frac{1}{2} (0 - log |\Sigma|) = - \frac{1}{2} log |\Sigma|$


-0.5 * unbiased_x.T = $- \frac{1}{2} (x-\mu_j)^T$

inv_cov = $\Sigma^{-1}$

unbiased_x = $(x- \mu_j)$

Se comprueba que la ecuación del programa es igual a la ecuación de la teoría

------------------------------------------------------------------

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.